# Evidence for water worlds around FGK stars
## Water-world means following Aguichine et al. (2021)

**Author: [Hannu Parviainen](mailto:hpparvi@gmail.com)** </br>
**Last edited: 25. October 2023**

Here we calculate the Bayesian evidences for our three water-world-population hypotheses using the Aguichine et al. (2021) water-rich-planet radius-density models to represent the water-world population density mean function.

The main difference with the Zeng et al. (2019) case is that we set an uniform prior, $U(0.1,1.0)$, on the water-world water fraction. This is because the Aguichine et al. (2021) models work slightly differently than the Zeng et al. (2019) models, and we don't need to fear that the water-rich planet population would be degenerate with rocky world population.

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from dynesty import NestedSampler
from uncertainties import ufloat

from spright import RMEstimator
from spright.io import read_tepcat
from spright.model import lnlikelihood_sample

%run common.py

In [3]:
names, radii, masses = read_tepcat('../../spright/data/TEPCat_FGK_20230522.csv')
rme = RMEstimator(nsamples=200, names=names, radii=radii, masses=masses, water='a21')
lpf = rme.lpf
rdm = lpf.rdm

In [5]:
def transform_h0(u):
    return transform_base(u, 0.0, 0.001, water_model='a21')

def transform_h1(u):
    return transform_base(u, 0.15, 0.5, water_model='a21')

def transform_h2(u):
    return transform_base(u, 0.5, 1.0, water_model='a21')

In [6]:
def lnlikelihood(pv):
    if is_puffy_population_ok(pv, lpf.rdm):
        lnl = lnlikelihood_sample(pv, lpf.density_samples, lpf.radius_samples,
                                  rdm._rr0, rdm._rdr, rdm._rx0, rdm._rdx, rdm.drocky, 
                                  rdm._wr0, rdm._wdr, rdm._wx0, rdm._wdx, rdm.dwater)
        return lnl if isfinite(lnl) else -1e6
    else:
        return -1e6

In [7]:
s0 = NestedSampler(lnlikelihood, transform_h0, 11, nlive=1000)
s1 = NestedSampler(lnlikelihood, transform_h1, 11, nlive=1000)
s2 = NestedSampler(lnlikelihood, transform_h2, 11, nlive=1000)

In [8]:
s0.run_nested()

12617it [02:47, 75.23it/s, +1000 | bound: 61 | nc: 1 | ncall: 318597 | eff(%):  4.288 | loglstar:   -inf < -264.547 <    inf | logz: -277.465 +/-  0.131 | dlogz:  0.001 >  1.009]


In [9]:
s1.run_nested()

15310it [03:49, 66.72it/s, +1000 | bound: 80 | nc: 1 | ncall: 403302 | eff(%):  4.054 | loglstar:   -inf < -262.204 <    inf | logz: -277.976 +/-  0.154 | dlogz:  0.002 >  1.009]


In [10]:
s2.run_nested()

14609it [04:16, 56.94it/s, +1000 | bound: 75 | nc: 1 | ncall: 379790 | eff(%):  4.121 | loglstar:   -inf < -263.243 <    inf | logz: -278.210 +/-  0.142 | dlogz:  0.001 >  1.009]


In [11]:
z0 = ufloat(s0.results.logz[-1], s0.results.logzerr[-1])
z1 = ufloat(s1.results.logz[-1], s1.results.logzerr[-1])
z2 = ufloat(s2.results.logz[-1], s2.results.logzerr[-1])

In [19]:
for v in (2*(z1 - z0), 2*(z2 - z0), 2*(z2 - z1)):
    print(str(v).replace('+/-', ' & '), end=' & ')

-1.0 & 0.7 & -1.5 & 0.7 & -0.5 & 0.7 & 

In [12]:
print(2*(z1 - z0), 2*(z2 - z0), 2*(z2 - z1))

-1.0+/-0.7 -1.5+/-0.7 -0.5+/-0.7


---

<center>
©2023 Hannu Parviainen
</center>